# AMATH 50 Workshop 1: Optimization
**Part II**

In this coding exercise, we try to program up

* Proximal Gradient Descent

algorithms use logistic regression as an example.

---

In [ ]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load data
train_images = np.load('data/train_images.npy')
train_labels = np.load('data/train_labels.npy')
test_images = np.load('data/test_images.npy')
test_labels = np.load('data/test_labels.npy')

num_train_images = train_labels.size
num_test_images = test_labels.size
image_size = 28*28

print('number of training images: %i' % num_train_images)
print('number of testing images: %i' % num_test_images)

## Proximal Gradient Descent

Now let us consider the strutural non-smooth optimization problem,
$$
\min_x f(x) := h(x) + g(x)
$$
where $h$ is a $\beta$-smooth convex function and $g$ is convex but not necessarily smooth.

The proximal gradient method can be summaried as follows,

* input: $x_0$
* set $k = 0$
* while $\beta\|x_k - x_{k-1}\| \ge $ tolerance
\begin{align*}
x_{k+1} &\leftarrow \text{prox}_{g/\beta}\left(x_k - \frac{1}{\beta} \nabla h(x_k)\right)\\
k &\leftarrow k + 1
\end{align*}
end
* output: $x_k$

Now let us consider the logistic regression with $\ell_1$ norm regularizer,
$$
h(x) = \sum_{i=1}^m [\ln(1 + \exp(\langle a_i, x \rangle)) - y_i\langle a_i, x \rangle], \quad
g(x) = \lambda \|x\|_1.
$$

In [ ]:
# define functions
A = train_images
y = train_labels

lam = 1.0

beta_h = 0.25*np.linalg.norm(A, 2)**2

def func_h(x):
    z = A.dot(x)
    return np.sum(np.log(1.0 + np.exp(z)) - y*z)

def grad_h(x):
    z = A.dot(x)
    return A.T.dot(np.exp(z)/(1.0 + np.exp(z)) - y)

def func_g(x):
    return lam*np.sum(np.abs(x))

def prox_g(x, t):
    t = lam*t
    y = np.zeros(x.size, dtype=x.dtype)

    ind = np.where(np.abs(x) > t)
    x_o = x[ind]

    y[ind] = np.sign(x_o)*(np.abs(x_o) - t)

    return y 

In [ ]:
# proximal gradient descent
def optimizeWithPGD(x0, func_h, func_g, grad_h, prox_g, beta_h, tol=1e-6, max_iter=1000):
    """
    Optimize with Proximal Gradient Descent Method
        min_x h(x) + g(x)
    where h is beta smooth and g is proxiable.
    
    input
    -----
    x0 : array_like
        Starting point for the solver
    func_h : function
        Input x and return the function value of h
    func_g : function
        Input x and return the function value of g
    grad_h : function
        Input x and return the gradient of h
    prox_g : function
        Input x and a constant float number and return the prox solution
    beta_h : float
        beta smoothness constant for h
    tol : float, optional
        Gradient tolerance for terminating the solver.
    max_iter : int, optional
        Maximum number of iteration for terminating the solver.
        
    output
    ------
    x : array_like
        Final solution
    obj_his : array_like
        Objective function value convergence history
    err_his : array_like
        Norm of gradient convergence history
    exit_flag : int
        0, norm of gradient below `tol`
        1, exceed maximum number of iteration
        2, others
    """
    # initial information
    x = x0.copy()
    g = grad_h(x)
    #
    step_size = 1.0/beta_h
    # not recording the initial point since we do not have measure of the optimality
    obj_his = np.zeros(max_iter)
    err_his = np.zeros(max_iter)
    
    # start iteration
    iter_count = 0
    err = tol + 1.0
    while err >= tol:
        # proximal gradient descent step
        x_new = prox_g(x - step_size*g, step_size)
        #
        # update information
        obj = func_h(x_new) + func_g(x_new)
        err = np.linalg.norm(x - x_new)/step_size
        #
        np.copyto(x, x_new)
        g = grad_h(x)
        #
        obj_his[iter_count] = obj
        err_his[iter_count] = err
        #
        # check if exceed maximum number of iteration
        iter_count += 1
        if iter_count >= max_iter:
            print('Proximal gradient descent reach maximum of iteration')
            return x, obj_his[:iter_count], err_his[:iter_count], 1
    #
    return x, obj_his[:iter_count], err_his[:iter_count], 0

In [ ]:
# apply the algorithm
x0 = np.zeros(image_size)

x, obj_his, err_his, exit_flag = optimizeWithPGD(x0, func_h, func_g, grad_h, prox_g, beta_h)

In [ ]:
# plot convergence result
fig, ax = plt.subplots(1, 2, figsize=(12,5))
ax[0].plot(obj_his)
ax[0].set_title('function value')
ax[1].semilogy(err_his)
ax[1].set_title('optimality condition')
fig.suptitle('Proximal Gradient Descent on Logistic Regression')

In [ ]:
# plot the identifier
plt.imshow(x.reshape(28, 28))

In [ ]:
# test result
A_test = test_images
y_test = test_labels

z_test = A_test.dot(x)

rate = np.sum(((z_test < 0.0) & (y_test == 0.0)) | ((z_test > 0.0) & (y_test == 1.0)))/num_test_images

print('correctly classify %0.2f of the testing data.' % rate)